In [2]:
import os

In [4]:
from pyspark.sql.functions import collect_list
from pyspark.sql import SparkSession
from pyspark.sql import Window
from pyspark.sql.functions import row_number, sum
from pyspark.sql.functions import col, rank

sparkSession = SparkSession.builder.enableHiveSupport().master("local[2]").getOrCreate()

data = sparkSession.read.parquet("/data/sample264")
meta = sparkSession.read.parquet("/data/meta")

In [1]:
t = data.groupBy(col('userId'),col('artistId')).count()
w = Window.partitionBy('userId').orderBy(col('count').desc())

t = t.withColumn('rank',rank().over(w))
t = t.filter(col('rank') < 100)

maxT = t.groupBy('userId').agg(sum('count').alias('sum_count'))

t = t.join(maxT,'userId','inner').withColumn('norm_count',col('count') / col('sum_count'))
t = t.orderBy(col('norm_count').desc(),col('userId'),col('artistId')).take(40)

for x in t:
    print("{} {}".format(x[0],x[1]))

NameError: name 'data' is not defined